In [34]:
import numpy as np
from roboticstoolbox.bin.rtbtool import panda
from scipy.spatial.transform import Rotation as R
from sympy import *
from sympy.physics.mechanics import ReferenceFrame

thetas = symbols('theta1:7')
lengths = symbols('l1:7')

# Function to create a 4x4 transformation matrix from a rotation matrix and translation vector
def transformation_matrix(rotation, translation):
    T = eye(4)
    T[0:3, 0:3] = rotation
    T[0:3, 3] = translation
    return T

links = [Matrix([0,0,lengths[i]]) for i in range(6)]


base = ReferenceFrame('base')
joints = [ReferenceFrame("joint"+str(i+1)) for i in range(6)]
joints[0].orient_axis(base, base.z, thetas[0])
joints[1].orient_axis(joints[0], joints[0].x, thetas[1])
joints[2].orient_axis(joints[1], joints[1].z, thetas[2])
joints[3].orient_axis(joints[2], joints[2].x, thetas[3])
joints[4].orient_axis(joints[3], joints[3].z, thetas[4])
joints[5].orient_axis(joints[4], joints[4].x, thetas[5])

# Create the transformation matrices
T1 = transformation_matrix(joints[0].dcm(base), links[0])
T2 = transformation_matrix(joints[1].dcm(joints[0]), links[1])
T3 = transformation_matrix(joints[2].dcm(joints[1]), links[2])
T4 = transformation_matrix(joints[3].dcm(joints[2]), links[3])
T5 = transformation_matrix(joints[4].dcm(joints[3]), links[4])
T6 = transformation_matrix(joints[5].dcm(joints[4]), links[5])

(T1 @ T2 @ T3 @ T4 @ T5 @ T6).subs({thetas[0]: 0, thetas[1]: np.pi/2, thetas[2]: 0, thetas[3]: 0, thetas[4]: 0, thetas[5]: 0, lengths[0]: 0, lengths[1]: 1, lengths[2]: 1, lengths[3]: 1, lengths[4]: 1, lengths[5]: 1})

Matrix([
[1,                    0,                    0,   0],
[0, 6.12323399573677e-17,                  1.0, 4.0],
[0,                 -1.0, 6.12323399573677e-17, 1.0],
[0,                    0,                    0,   1]])

In [16]:
import roboticstoolbox as rtb
import matplotlib.pyplot as plt
from itertools import product


puma = rtb.models.DH.Puma560()
qlim = puma.qlim
angles = np.array([np.linspace(qlim[0, i], qlim[1, i], 10) for i in range(0, puma.q.size)])

workspace_points = []

for angle_set in product(*angles):
    T = puma.fkine(angle_set)  # Forward kinematics
    position = T.t           # Extract translation part
    workspace_points.append(position)

workspace_points = np.array(workspace_points)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(workspace_points[:, 0], workspace_points[:, 1], workspace_points[:, 2], s=1)
ax.set_title('End Effector Workspace')
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
plt.show()

In [19]:
# Function to create a 4x4 transformation matrix from a rotation matrix and translation vector
def transformation_matrix2(rotation, translation):
    T = np.eye(4)
    T[0:3, 0:3] = rotation
    T[0:3, 3] = translation
    return T

# Example rotation matrices (identity matrices for simplicity)
R1 = R.from_euler("z", 0, degrees=True).as_matrix()  # Rotation about z-axis for joint 1
R2 = R.from_euler("x", 45, degrees=True).as_matrix()  # Rotation about x-axis for joint 2
R3 = np.eye(3)  # Identity rotation matrix for joint 3
# Example translation vectors (position of joint 1 and joint 2 relative to the previous joint)
P1 = np.array([0, 0, 0])  # Translation along z-axis for joint 1
P2 = np.array([0, 0, 1])  # Translation along z-axis for joint 2
P3 = np.array([0, 0, np.sqrt(2)])  # Translation along z-axis for joint 3

# Create the transformation matrices
T1 = transformation_matrix2(R1, P1)
T2 = transformation_matrix2(R2, P2)
T3 = transformation_matrix2(R3, P3)

# Compute the end effector transformation matrix
T_end_effector = T1 @ T2 @ T3

# Extract the position of the end effector (last column of the 4x4 matrix)
end_effector_position = T_end_effector[0:3, 3]

# Output the end effector position
print("End Effector Position:", end_effector_position)

End Effector Position: [ 0. -1.  2.]
